In [43]:
pip install pyvi

#Import library

In [44]:
import warnings
import numpy as np
from scipy.special import logsumexp
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_is_fitted, check_non_negative
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils.validation import _check_sample_weight
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import check_array
from sklearn.utils.multiclass import _check_partial_fit_first_call
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics import classification_report
from pyvi import ViTokenizer

#Load data

In [45]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/NLP/dataset_final_nlpp - dataset_final_nlpp.csv")

In [46]:
data = data.dropna()

In [47]:
data.head()

,review,label
0,shop quá tệ đặt màu trắng giao màu xanh,0
1,mình rất hài lòng với sản phẩm của shop\nhàng ...,1
2,hình ảnh chỉ mang tính chất nhận xu hàng về nh...,1
3,sản phẩm thì như hình minh họa giao hàng nhanh...,1
4,sản phẩm chất lương đã mua cái thứ 2 rẻ hơn cá...,1


In [48]:
sentences = data['review']

In [49]:
label_counts = data['label'].value_counts()

In [50]:
print(label_counts)

1    4128
0    3759
2    1337
Name: label, dtype: int64


#Preprocessing

In [51]:
# Chuẩn hóa văn bản
data['review'] = data['review'].str.lower()
data['review'] = data['review'].replace('[^\w\s]', '', regex=True)


In [52]:
data.head()

,review,label
0,shop quá tệ đặt màu trắng giao màu xanh,0
1,mình rất hài lòng với sản phẩm của shop\nhàng ...,1
2,hình ảnh chỉ mang tính chất nhận xu hàng về nh...,1
3,sản phẩm thì như hình minh họa giao hàng nhanh...,1
4,sản phẩm chất lương đã mua cái thứ 2 rẻ hơn cá...,1


In [53]:
X = data['review']
y = data['label']

In [54]:
X.shape

(9224,)

In [55]:
import re
def chuyen_viet_tat(s):

    tu_dien_viet_tat = {
    r'\bmn\b': 'mọi người',
    r'\bsp\b': 'sản phẩm',
    r'\bđc\b': 'được',
    r'\bnhah\b': 'nhanh',
    r'\bhong\b': 'không',
    r'\bk\b': 'không',
    r'\bthui\b': 'thôi',
    r'\be\b': 'em',
    r'\bgh\b': 'giao hàng',
    r'\bnhư v\b': 'như vậy',
    r'\btr\b': 'trời',
    r'\bon\b': 'ổn',
    r'\bnma\b': 'nhưng mà',
    r'\bnm\b': 'nhưng mà',
    r'\bkhéc\b': 'khét',
    r'\bhih\b': 'hình',
    r'\bqc\b': 'quảng cáo',
    r'\bt2\b': 'thứ hai',
    r'\bsd\b': 'sử dụng',
    r'\bhcm\b': 'hồ chí minh',
    r'\bhn\b': 'hà nội',
    r'\bcmt\b': 'comment',
    r'\bng\b': 'người',
    r'\bsr\b': 'xin lỗi',
    r'\blg\b': 'lượng',
    r'\bhsd\b': 'hạn sử dụng',
    r'\bhok\b': 'không',
    r'\bh\b': 'giờ',
    r'\bae\b': 'anh em',
    r'\bnt\b': 'nhắn tin',
    r'\bsop\b': 'shop',
    r'\bln\b': 'luôn',
    r'\btrc\b': 'trước',
    r'\bthik\b': 'thích',
    r'\bmk\b': 'mình',
    r'\bsx\b': 'sản xuất',
    r'\bnhug\b': 'nhưng',
    r'\bsl\b': 'số lượng',
    r'\bthank\b': 'cảm ơn',
    r'\bthanks\b': 'cảm ơn',
    r'\bnka\b': 'nha',
    r'\bthowm\b': 'thơm',
    r'\bchuwa\b': 'chưa',
    r'\bbic\b': 'biết',
    r'\bgood\b': 'tốt',
    r'\bgod\b': 'tốt',
    r'\btn\b': 'tin nhắn',
    r'\bđươcj\b': 'được',
    r'\bvs\b': 'với',
    r'\bdk\b': 'được',
    r'\br\b': 'rồi',
    r'\bmjh\b': 'mình',
    r'\bntn\b': 'như thế nào',
    r'\bbit\b': 'biết',
    r'\bkm\b': 'khuyến mãi',
    r'\bmng\b': 'mọi người',
    r'\bcx\b': 'cũng',
    r'\btt\b': 'tiếp tục',
    r'\bcl\b': 'chất lượng',
    r'\bktra\b': 'kiểm tra',
    r'\bktr\b': 'kiểm tra',
    r'\bz\b': 'vậy',
    r'\bb\b': 'bạn',
    r'\bkbt\b': 'không biết',
    r'\bj\b': 'gì',
    r'\bđt\b': 'điện thoại',
    r'\blụong\b': 'lượng',
    r'\bkaij\b': 'lại',
    r'\bhk\b': 'không',
    r'\bko\b': 'không',
    r'\bđưj\b': 'được',
    r'\bbidy\b': 'body',
    r'\boki\b': 'ok',
    r'\bnv\b': 'nhân viên',
    r'\bchx\b': 'chưa',
    r'\bbt\b': 'biết',
    r'\bnhm\b': 'nhưng mà',
    r'\bwa\b': 'quá',
    r'\bchâm\b': 'chậm',
    r'\bkhum\b': 'không',
    r'\bchac\b': 'chắc',
    r'\bcjhâm\b': 'chậm',
    r'\bahop\b': 'shop',
    r'\bxme\b': 'xem',
    r'\bđnahs\b': 'đánh',
    r'\bnghũ\b': 'nghĩ',
    r'\btrỏng\b': 'trong',
    r'\bokela\b': 'ok',
    r'\btks\b': 'cảm ơn',
    r'\bcùa\b': 'của',
    r'\bơt\b': 'ở',
    r'\bksao\b': 'không sao',
    r'\bokelah\b': 'ok',
    r'\bmụi\b': 'mọi',
    r'\bngừoi\b': 'người',
    r'\bmac\b': 'mác',
    r'\bhfgng\b': 'hàng',
    r'\bgg\b': 'giảm giá',
    r'\bap\b': 'áp',
    r'\bdc\b': 'được',
    r'\bmag\b': 'mang',
    r'\bbs\b': 'bác sĩ',
    r'\bthươgf\b': 'thường',
    r'\btu van\b': 'tư vấn',
    r'\btgian\b': 'thời gian',
    r'\bch\b': 'chưa',
    r'\btg\b': 'thời gian',
    r'\bcbi\b': 'chuẩn bị',
    r'\bngưồi\b': 'người',
    r'\bnhê\b': 'nha',
    r'\bni\b': 'này',
    r'\b\fai\b': "phải",
     r'\bship pơ\b': "shipper",
    r'\bhiêun\b': 'hiệu',
    r'\btố\b': 'tốt',
    r'\btuýt\b': 'tuýp',
    r'\bnx\b': 'nữa',
    r'\btw\b': 'trung ương',
    r'\blm\b': 'làm',
    r'\bqq\b': 'quần què',
    r'\bzire\b': 'size',
    r'\bđâh\b': 'đâu',
    r'\blun\b': 'luôn',
    r'\bphake\b': 'fake',
    r'\blonn\b': 'lol',
    r'\bchổ\b': 'chỗ',
    r'\bquâ\b': 'quá',
    r'\bsít\b': 'shit',
    r'\bdb\b': 'đầu_buồi',
    r'\bchày\b': 'trầy',
    r'\bqa\b': 'quá',
    r'\bns\b': 'nó',
    r'\bbôg\b': 'bông',
    r'\bxâu\b': 'xấu',
    r'\blh\b': 'liên hệ',
    r'\bll\b': 'liên lạc',
    r'\bgioa\b': 'giao',
    r'\bnheng\b': 'nhưng',
    r'\bak\b': 'á',
    r'\bphẩn\b': 'phẩm',
    r'\bcám\b': 'cảm',
    r'\bah\b': 'á',
    r'\bokey\b': 'ok',
    r'\blứm\b': 'lắm',
    r'\btoẹt\b': 'tuyệt',
    r'\bkím\b': 'kiếm',
    r'\bgood\b': 'tốt',
    r'\btq\b': 'trung quốc',
    r'\btiệc vời\b': 'tuyệt vời',
    r'\bcunhx\b': 'cũng',
    r'\bm\b': 'mình',
    r'\bspham\b': 'sản phẩm',
    r'\bks\b': 'không sao',
    r'\btieenf\b': 'tiền',
    r'\bqt\b': 'quốc tế',
    r'\bnhunge\b': 'nhưng',
    r'\bphaamf\b': 'phẩm',
    r'\bbk\b': 'biết',
    r'\bcos\b': 'có',
    r'\btot\b': 'tốt',
    r'\bkh\b': 'khách hàng',
    r'\bnod\b': 'nó',
    r'\brui\b': 'rồi',
    r'\btrk\b': 'trước',
    r'\bkg\b': 'không',
    r'\bđx\b': 'được',
    r'\bđưocj\b': 'được',
    r'\blzd\b': 'lazada',
    r'\bshipper\b': 'shiper',
    r'\bnhưnh\b': 'nhưng',
    r'\bnhung\b': 'nhưng',
    r'\bokie\b': 'ok',
    r'\boce\b': 'ok',
    r'\bb\b': 'bạn',
    r'\bph\b': 'phẩm',
    r'\bsez\b': 'sẽ',
    r'\bđng\b': 'đóng',
    r'\bcb\b': 'chuẩn bị',
    r'\bđươ\b': 'được',
    r'\blượg\b': 'lượng',
    r'\bđươc\b': 'được',
    r'\batvs\b': 'an toàn vệ sinh',
    r'\bgđ\b': 'gia đình',
    r'\btc\b': 'tính chất',
    r'\bđonga\b': 'đóng',
    r'\bđings\b': 'đóng',
    r'\bnhueng\b': 'nhưng',
    r'\bre\b': 'rẻ',
    r'\brats\b': 'rất',
    r'\btyueet\b': 'tuyệt',
    r'\bđưôcj\b': 'được',
    r'\bqả\b': 'quả',
    r'\bddep\b': 'đẹp',
    r'\bkl\b': 'không',
    r'\bnnao\b': 'như nào',
    r'\bhủ\b': 'hũ',
    r'\b\n\b': " ",
    r'\bgood\b': "tốt",
    r'\bbt\b': "bình thường",
    r'\bbth\b': "bình thường",
    r'\bbthg\b': "bình thường",

    }

    for viet_tat, viet_thuong in tu_dien_viet_tat.items():
        s = re.sub(viet_tat, viet_thuong, s)
    return s

In [56]:
with open('/content/drive/MyDrive/NLP/stopword.txt', 'r', encoding='utf-8') as file:
    stopword_list = file.read().splitlines()

def remove_stopwords(sentence):
    words = sentence.split()
    filtered_words = [word for word in words if word.lower() not in stopword_list]
    return ' '.join(filtered_words)


In [57]:
import re

def loai_bo_chu_cai_lap(text):
    result = re.sub(r'(\w)\1*', r'\1', text)
    return result


In [58]:
X_str = [chuyen_viet_tat(str(x)) for x in X]
X_replace = [loai_bo_chu_cai_lap(x) for x in X_str]
X_new = [remove_stopwords(sentence) for sentence in X_replace]

#Tokenization

In [59]:
from pyvi import ViTokenizer
X_tokenize = [ViTokenizer.tokenize(review) for review in X_new]

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_tokenize,y,train_size = 0.8, random_state=42)

In [61]:
data['review'] = X_tokenize

In [62]:
data.to_csv("data_preprocessing.csv")

In [63]:
vectorizer =  TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

#Implement Multinomial Naive Bayes

In [64]:
class MultinomialNBB(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=1.0, fit_prior=True, class_prior=None):
        self.alpha = alpha
        self.fit_prior = fit_prior
        self.class_prior = class_prior

    def count(self, X, Y):
        self.feature_count_ += safe_sparse_dot(Y.T, X)
        self.class_count_ += Y.sum(axis=0)

    def cal_likelihood(self, alpha):
        smoothed_fc = self.feature_count_ + alpha
        smoothed_cc = smoothed_fc.sum(axis=1)

        self.feature_log_prob_ = np.log(smoothed_fc/smoothed_cc.reshape(-1, 1))

    def cal_prior(self, class_prior=None):
        n_classes = len(self.classes_)
        if class_prior is not None:
            self.class_log_prior_ = np.log(class_prior)
        elif self.fit_prior:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", RuntimeWarning)
                log_class_count = np.log(self.class_count_)
            self.class_log_prior_ = log_class_count - np.log(self.class_count_.sum())
        else:
            self.class_log_prior_ = np.full(n_classes, -np.log(n_classes))

    def init_counters(self, n_classes, n_features):
        self.class_count_ = np.zeros(n_classes, dtype=np.float64)
        self.feature_count_ = np.zeros((n_classes, n_features), dtype=np.float64)

    def fit(self, X, y, sample_weight=None):
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_

        if sample_weight is not None:
            Y = Y.astype(np.float64, copy=False)
            sample_weight = _check_sample_weight(sample_weight, X)
            sample_weight = np.atleast_2d(sample_weight)
            Y *= sample_weight.T

        class_prior = self.class_prior
        n_classes = Y.shape[1]
        self.init_counters(n_classes, n_features)
        self.count(X, Y)
        alpha = self.alpha
        self.cal_likelihood(alpha)
        self.cal_prior(class_prior=class_prior)
        return self

    def joint_log_likelihood(self, X):
        return safe_sparse_dot(X, self.feature_log_prob_.T) + self.class_log_prior_

    def predict(self, X):
        jll = self.joint_log_likelihood(X)
        return self.classes_[np.argmax(jll, axis=1)]


In [65]:
model =  MultinomialNBB()
model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86       735
           1       0.79      0.94      0.85       859
           2       0.83      0.08      0.14       251

    accuracy                           0.81      1845
   macro avg       0.81      0.64      0.62      1845
weighted avg       0.81      0.81      0.76      1845

